In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from torch import nn
import itertools

In [116]:
class MyNeuralNetwork(nn.Module):
    def __init__(self, l1 = 100) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, l1),
            nn.LeakyReLU(),
            nn.Linear(l1, 7)
        )
    
    def forward(self, inputs):
        return self.net(inputs)

In [117]:
class MultiEmoVA(Dataset):
    def __init__(self, data_path) -> None:
        super().__init__()
        data=pd.read_csv(data_path)
        if 'emotion' in data:
            self.inputs = torch.tensor(data.drop("emotion", axis=1).to_numpy(np.float32))
            self.index2label = [label for label in data["emotion"].unique()]
            label2index = {label: i for i, label in enumerate(self.index2label)}
            self.labels = torch.tensor(data["emotion"].apply(lambda x: label2index[x]))
        else:
            self.inputs = torch.tensor(data.to_numpy(np.float32))
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        if hasattr(self, 'labels'):
            return self.inputs[index], self.labels[index]
        else:
            return self.inputs[index]
    
    @staticmethod
    def label_dict():
        data=pd.read_csv("../dataset/dataset.csv")
        return [label for label in data["emotion"].unique()]
        

In [132]:
def hyper_model(l1, learn):
    device = "cuda"
    data = MultiEmoVA("../dataset/dataset.csv")
    unlabeled = MultiEmoVA("../dataset/test_to_submit.csv")

    train, val, test = random_split(data, [0.7, 0.2, 0.1])

    model = MyNeuralNetwork(l1)


    train_loader = DataLoader(train)
    val_loader = DataLoader(val)
    test_loader = DataLoader(test)

    loss_fn = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(model.parameters(), lr = learn)

    n_epochs = 20

    for epoch in range(n_epochs):
        for inputs, labels in train_loader:
            output = model(inputs)
            loss = loss_fn(output, labels)
            loss.backward()
            optim.step()
            optim.zero_grad()

        correct = 0
        for inputs, labels in val_loader:
            output = model(inputs)
            correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
        #print(f"Validation accuracy is: {correct/len(val)}")


    correct = 0
    for inputs, labels, in test_loader:
        output = model(inputs)
        correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
    print(f"Hidden nodes: {l1}, Learning rate {learn}")
    print(f"Test accuracy is: {correct/len(test)}")


    unlabeled_loader = DataLoader(unlabeled)
    out = pd.DataFrame(list(map(lambda x: MultiEmoVA.label_dict()[model(x).softmax(dim=1).argmax(dim=1)], unlabeled_loader)))
    out.to_csv('../processed/outputs', index = False, header = False)


In [133]:
hyperparams = itertools.product([100, 150, 200, 250], [0.001, 0.005, 0.01, 0.05])
list(map(lambda x: hyper_model(*x), hyperparams))


Hidden layers: 100, Learning rate 0.001
Test accuracy is: 0.6724137663841248
Hidden layers: 100, Learning rate 0.005
Test accuracy is: 0.5431034564971924
Hidden layers: 100, Learning rate 0.01
Test accuracy is: 0.6034482717514038
Hidden layers: 100, Learning rate 0.05
Test accuracy is: 0.47413793206214905
Hidden layers: 150, Learning rate 0.001
Test accuracy is: 0.5517241358757019
Hidden layers: 150, Learning rate 0.005
Test accuracy is: 0.5603448152542114
Hidden layers: 150, Learning rate 0.01
Test accuracy is: 0.6379310488700867
Hidden layers: 150, Learning rate 0.05
Test accuracy is: 0.4655172526836395
Hidden layers: 200, Learning rate 0.001
Test accuracy is: 0.6379310488700867
Hidden layers: 200, Learning rate 0.005
Test accuracy is: 0.5344827771186829
Hidden layers: 200, Learning rate 0.01
Test accuracy is: 0.568965494632721
Hidden layers: 200, Learning rate 0.05
Test accuracy is: 0.3965517282485962
Hidden layers: 250, Learning rate 0.001
Test accuracy is: 0.6465517282485962
Hidde

list

In [134]:
#Generate outputs using best hyperparameters
hyper_model(100, 0.001)

Hidden layers: 100, Learning rate 0.001
Test accuracy is: 0.6982758641242981


tensor(0.6983)